In [15]:
import numpy as np
import cv2
import math
from numba import cuda, int32

In [16]:
#Salt&Pepper
def add_noise(image, probability):
    noisy_image = image.copy()
    random_matrix = np.random.rand(image.shape[0], image.shape[1])
    noisy_image[random_matrix < probability] = 0
    noisy_image[random_matrix > 1 - probability] = 255
    return noisy_image

In [17]:
#cuda_meidan
@cuda.jit
def median_filter_kernel(input_image, output_image):
    i, j = cuda.grid(2)
    if i >= 1 and j >= 1 and i < input_image.shape[0] - 1 and j < input_image.shape[1] - 1:
        local_window = cuda.local.array(shape=9, dtype=np.uint8)

        idx = 0
        for di in range(-1, 2):
            for dj in range(-1, 2):
                local_window[idx] = input_image[i + di, j + dj]
                idx += 1

        for k in range(8):
            for l in range(8 - k):
                if local_window[l] > local_window[l + 1]:
                    local_window[l], local_window[l + 1] = local_window[l + 1], local_window[l]

        output_image[i, j] = local_window[4]

In [18]:
#CPU_median
def apply_median_filter_cpu(input_image):
    output_image = np.zeros_like(input_image)
    start_time = cv2.getTickCount()

    for i in range(1, input_image.shape[0] - 1):
        for j in range(1, input_image.shape[1] - 1):
            local_window = input_image[i - 1:i + 2, j - 1:j + 2].flatten()
            local_window.sort()
            output_image[i, j] = local_window[4]

    end_time = cv2.getTickCount()
    elapsed_time_s = (end_time - start_time) / cv2.getTickFrequency()
    return output_image, elapsed_time_s

In [19]:
def main():
    input_image = cv2.imread('input.bmp', cv2.IMREAD_GRAYSCALE)

    noisy_image = add_noise(input_image, probability=0.05)
    cv2.imwrite('noisy_image.bmp', noisy_image)

    gpu_filtered_image, gpu_time = apply_median_filter_gpu(noisy_image)
    cv2.imwrite('gpu_filtered_image.bmp', gpu_filtered_image)

    cpu_filtered_image, cpu_time = apply_median_filter_cpu(noisy_image)
    cv2.imwrite('cpu_filtered_image.bmp', cpu_filtered_image)

    print(f"GPU processing time: {gpu_time:.6f} seconds")
    print(f"CPU processing time: {cpu_time:.6f} seconds")

In [20]:
if __name__ == '__main__':
    main()

GPU processing time: 0.364122 seconds
CPU processing time: 0.835027 seconds
